In [1]:
import pandas as pd

### krx 사이트에 등록되어 있는 상장사 정보(html) 다운로드 받아 dataframe 객체 생성
* krx 상장사 정보 url : http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13

In [2]:
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

In [3]:
code_df.head(5)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
1,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
2,GS글로벌,1250,상품 종합 도매업,"수출입업(시멘트,철강금속,전기전자,섬유,기계화학),상품중개,광업,채석업/하수처리 서...",1976-06-26,12월,김태형,http://www.gsgcorp.com,서울특별시
3,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"권순호, 정경구",http://www.hdc-dvp.com,서울특별시
4,LG이노텍,11070,전자부품 제조업,기타 전자부품 제조업,2008-07-24,12월,정철동,http://www.lginnotek.co.kr,서울특별시


### Q1. 상장사 정보 중 종목코드를 왼쪽이 '0' 으로 채워진 6자리 값으로 변환하시오

In [4]:
code_df['종목코드'] = code_df['종목코드'].map('{:06d}'.format)

### Q2. code_df 객체에서 '회사명', '종목코드' 컬러만 선택하여 code_df 객체에 재 할당 하시오.

In [5]:
code_df = code_df[['회사명', '종목코드']]

In [6]:
code_df.head()

,회사명,종목코드
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


### Q3. code_df 객체에서 '회사명' , '종목코드' 컬럼의 이름을 각 각 'name', 'code' 로 변경 하시오

In [7]:
code_df = code_df.rename(columns={'회사명' : 'name',
                                  '종목코드' : 'code'})

In [8]:
code_df.head()

,name,code
0,DSR,155660
1,GS,078930
2,GS글로벌,001250
3,HDC현대산업개발,294870
4,LG이노텍,011070


### Q4. get_url 함수를 정의하시오
* 입럭받은 '회사명'(item_name) 정보와 code_df 객체를 이용하여 '상장 코드' 를 이용하여 naver 주식 시세 정보를 검색하는 url을 작성하시오
* naver 주식 시세 정보 검색 url : https://finance.naver.com/item/sise_day.nhn?code=`상장 코드`
* 작성한 함수를 이용하여 삼성전자 주식 조회를 위한 naver 주식 시세 정보 검색 url 정보를 출력하시오

In [9]:
def get_url(item_name, code_df):
  code = code_df['code'][code_df['name'] == item_name].values[0]
  url = 'https://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)

  print("request URL = {}".format(url))
  return url

In [10]:
item_name='삼성전자'
url = get_url(item_name, code_df)

request URL = https://finance.naver.com/item/sise_day.nhn?code=005930


### url을 이용한 주식시세 정보 호출을 위한 requests 라이브러리 임포트

In [11]:
import requests

### Q5. Q4에서 생성한 naver 주식 시세정보 조회 url을 이용하여 200일 간의 삼성전자의 주식 시세 정보를 데이터프레임 객체에 저장하시오

In [12]:
df = pd.DataFrame()

priceList = []

for page in range(1, 21):
  pg_url = '{url}&page={page}'.format(url=url, page=page)
  df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

In [13]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.06.25,51900.0,1000.0,52100.0,53000.0,51900.0,18350336.0
2,2020.06.24,52900.0,1500.0,51900.0,53900.0,51600.0,24519552.0
3,2020.06.23,51400.0,600.0,52500.0,52800.0,51100.0,18086152.0
4,2020.06.22,52000.0,900.0,52000.0,52600.0,51800.0,13801350.0


### Q6. dataframe 객체의 컬럼이름을 영문으로 변경
* 컬럼의 이름은 'date', 'close', 'diff', 'open', 'high', 'low', 'volume' 롤 지정

In [14]:
df = df.rename(columns= {'날짜': 'date',
                         '종가': 'close',
                         '전일비': 'diff',
                         '시가': 'open',
                         '고가': 'high',
                         '저가': 'low',
                         '거래량': 'volume'})

In [15]:
df.head()

,date,close,diff,open,high,low,volume
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020.06.25,51900.0,1000.0,52100.0,53000.0,51900.0,18350336.0
2,2020.06.24,52900.0,1500.0,51900.0,53900.0,51600.0,24519552.0
3,2020.06.23,51400.0,600.0,52500.0,52800.0,51100.0,18086152.0
4,2020.06.22,52000.0,900.0,52000.0,52600.0,51800.0,13801350.0


### Q7. df 객체에서 Nan 제거

In [16]:
df = df.dropna()

In [17]:
df.head()

,date,close,diff,open,high,low,volume
1,2020.06.25,51900.0,1000.0,52100.0,53000.0,51900.0,18350336.0
2,2020.06.24,52900.0,1500.0,51900.0,53900.0,51600.0,24519552.0
3,2020.06.23,51400.0,600.0,52500.0,52800.0,51100.0,18086152.0
4,2020.06.22,52000.0,900.0,52000.0,52600.0,51800.0,13801350.0
5,2020.06.19,52900.0,600.0,52600.0,52900.0,51600.0,18157985.0


### df 객체에 각 원소들의 값이 object로 되어 있기 때문에 분석을 위해 float type으로 

In [18]:
df.dtypes

date       object
close     float64
diff      float64
open      float64
high      float64
low       float64
volume    float64
dtype: object

In [19]:
import locale
from locale import atof

locale.setlocale(locale.LC_NUMERIC, '')

df['date'] = pd.to_datetime(df['date'])

df[['close', 'diff', 'open', 'high', 'low', 'volume']] = \
df[['close', 'diff', 'open', 'high', 'low', 'volume']].applymap(atof)
df = df.sort_values(by=['date'], ascending=True)

AttributeError: ignored

In [20]:
df.dtypes

date      datetime64[ns]
close            float64
diff             float64
open             float64
high             float64
low              float64
volume           float64
dtype: object

In [21]:
df.head()

,date,close,diff,open,high,low,volume
1,2020-06-25,51900.0,1000.0,52100.0,53000.0,51900.0,18350336.0
2,2020-06-24,52900.0,1500.0,51900.0,53900.0,51600.0,24519552.0
3,2020-06-23,51400.0,600.0,52500.0,52800.0,51100.0,18086152.0
4,2020-06-22,52000.0,900.0,52000.0,52600.0,51800.0,13801350.0
5,2020-06-19,52900.0,600.0,52600.0,52900.0,51600.0,18157985.0


### 삼성 전자 종가 기준 일별 주가 시계열 차트 조회

In [22]:
# 시계열 차트 생성을 위한 라이브러리 임포트
import plotly.graph_objects as go

In [23]:
# 차트 생성을 위한 데이터 생성
trace = go.Scatter(x=df['date'],
                   y=df['close'],
                   name=item_name)

data = [trace]

In [24]:
# 시계열 차트의 레이웃 정보 생성
layout = dict(
    title='{}의 종가(close) Time Series'.format(item_name),
    # x축 데이터 타입 지정(date)
    xaxis = dict(
        rangeslider=dict(),
        type='date'
        ),
)

In [25]:
# 차트 객체 생성 및 조회
fig = go.Figure(data=data, layout=layout)
fig.show()